In [1]:
import pickle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
import numpy as np
import csv
import pandas as pd



# Load the CSV file into a pandas DataFrame
train_df = pd.read_csv('mnist_train.csv')
test_df = pd.read_csv('mnist_test.csv')

# Extract labels and features
train_labels = train_df.iloc[:, 0]  # the label is in the first column
train_features = train_df.iloc[:, 1:]  
test_labels = test_df.iloc[:, 0]  # Assuming the label is in the first column
test_features = test_df.iloc[:, 1:]  

# Convert labels and features into numpy arrays
train_labels_array = train_labels.to_numpy()
train_features_array = train_features.to_numpy()
test_labels_array = test_labels.to_numpy()
test_features_array = test_features.to_numpy()


In [2]:
print(len(train_labels_array))
print(len(test_labels_array))
print(type(train_features_array))

print(len(train_features_array[0]))

60000
10000
<class 'numpy.ndarray'>
784


In [3]:

deg_l1=9
deg_l2=8
deg_data=[]
deg_labels=[]

new_train_data=[]
new_train_lab=[]
print("removing labels 8 9 from train...")
for i in range(len(train_labels_array)):
    if train_labels_array[i]==deg_l1 or train_labels_array[i]==deg_l2:
        deg_data.append(train_features_array[i])
        deg_labels.append(train_labels_array[i])
    else:
        new_train_data.append(train_features_array[i])
        new_train_lab.append(train_labels_array[i])
        
new_test_data=[]
new_test_lab=[]
print("removing labels 8 9 from test...")
for i in range(len(test_labels_array)):
    if test_labels_array[i]==deg_l1 or test_labels_array[i]==deg_l2:
        deg_data.append(test_features_array[i])
        deg_labels.append(test_labels_array[i])
    else:
        new_test_data.append(test_features_array[i])
        new_test_lab.append(test_labels_array[i])
        
print(len(deg_data))
print(len(new_train_data))
print(len(new_test_data))

removing label 9 from train...
removing label 9 from test...
13783
48200
8017


In [4]:
from sklearn.model_selection import train_test_split

# Create data splits
rp=13/50
print(len(new_train_data))
print(len(new_train_lab))
nn_tr_data,new_unseen_data, nn_tr_lab ,new_unseen_lab=train_test_split(new_train_data, new_train_lab, test_size=rp, stratify=new_train_lab, random_state=42)


print(len(new_unseen_data))
print(len(nn_tr_data))

48200
48200
12532
35668


In [5]:
import random

# Balance datasets

final_train_data=[]
final_train_lab=[]
final_test_data=[]
final_test_lab=[]
final_unseen_data=[]
final_unseen_lab=[]
final_deg_data=[]
final_deg_lab=[]
deg_label=8
c=0
for i in range(35000):
        final_train_data.append(nn_tr_data[i])
        final_train_lab.append(nn_tr_lab[i])

for i in range(12000):
        final_unseen_data.append(new_unseen_data[i])
        final_unseen_lab.append(new_unseen_lab[i])    
        
for i in range(8000):
        final_test_data.append(new_test_data[i])
        final_test_lab.append(new_test_lab[i]) 

c=50
for i in range(13000):
    try:
        final_deg_data.append(deg_data[i])
        final_deg_lab.append(deg_label)
    except Exception as e:
        final_deg_data.append(deg_data[i-c])
        final_deg_lab.append(deg_label)
        c=c+1

print(len(final_train_lab))
print(len(final_train_data))
print(len(final_test_lab))
print(len(final_unseen_lab))
print(len(final_deg_lab))

35000
35000
8000
12000
13000


In [6]:
def image_conv(imgs_array):

    x=[]
    c=0
    for i in range(len(imgs_array)):
        try:
            rgb_image = np.stack((imgs_array[i],) * 3, axis=-1)
            img=np.array(rgb_image).reshape((28,28,3))
            x.append(img)

            
        except Exception as e:
            print("An exception has occurred")

    return x

# Store the csv values in numpy arrays
print(final_train_data[0].shape)
ult_train_data=image_conv(final_train_data)
ult_test_data=image_conv(final_test_data)
ult_unseen_data=image_conv(final_unseen_data)
ult_deg_data=image_conv(final_deg_data)

(784,)
exceptions
0
exceptions
0
exceptions
0
exceptions
0


In [8]:
from transformers import ViTModel
from transformers.modeling_outputs import SequenceClassifierOutput
import torch.nn.functional as F
import numpy as np
import torch

import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image

# Define the model

class ViTForImageClassification(nn.Module):
    def __init__(self, num_labels=8):
        super(ViTForImageClassification, self).__init__()
        self.vit = ViTModel.from_pretrained('google/vit-base-patch16-224')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.vit.config.hidden_size, num_labels)
        self.num_labels = num_labels

    def forward(self, pixel_values, labels):
        outputs = self.vit(pixel_values=pixel_values)
        output = self.dropout(outputs.last_hidden_state[:,0])
        logits = self.classifier(output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        if loss is not None:
            return logits, loss.item()
        else:
            return logits, None
    
    
    
    def emb_extr_new_v1(self, pixel_values):
        
        outputs = self.vit(pixel_values=pixel_values)
        
        return outputs.last_hidden_state[:,0]
    
    
    


In [9]:
# Transform the data from numpy array to images
def data_into_format(data, labels):
    formatted_data=[]
    for i in range(len(data)):
        formatted_data.append((data[i][0], labels[i]))

    
    return formatted_data
 
# Convert images into tensors
transform_2 = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit the input size of the transformer
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize images
])

print("Train data")
f_train_data=[]
for i in range(len(ult_train_data)):
    img_pil = Image.fromarray(np.uint8(ult_train_data[i]))
    f_train_data.append(img_pil)

print("Test data")
f_test_data=[]
for i in range(len(ult_test_data)):
    img_pil = Image.fromarray(np.uint8(ult_test_data[i]))
    f_test_data.append(img_pil)

print("New unseen data")
f_unseen_data=[]
for i in range(len(ult_unseen_data)):
    img_pil = Image.fromarray(np.uint8(ult_unseen_data[i]))
    f_unseen_data.append(img_pil)


print("Drifted data")
f_deg_data=[]
for i in range(len(ult_deg_data)):
    img_pil = Image.fromarray(np.uint8(ult_deg_data[i]))
    f_deg_data.append(img_pil)
    


train
test
unseen
deg


In [10]:
import torch
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


# Initialize model, loss function, and optimizer

#BATCH_SIZE = 100
EPOCHS=3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model=ViTForImageClassification()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
   

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import ViTFeatureExtractor
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch


class CustomDataset(Dataset):
    def __init__(self, images, labels, batch_size, transform=None):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.transform = transform

    def __len__(self):
        return len(self.images) // self.batch_size

    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = (idx + 1) * self.batch_size
        
        batch_images = self.images[start_idx:end_idx]
        batch_labels = self.labels[start_idx:end_idx]
        
        # Check if batch is empty
        if not batch_images:
            raise ValueError("Empty batch. Please check the dataset and batch size.")
        
        if self.transform:
            batch_images = [self.transform(image) for image in batch_images]
        else:
            batch_images = [transforms.ToTensor()(image) for image in batch_images]
        
        # Convert label to tensor
        batch_labels = torch.tensor(batch_labels, dtype=torch.long)
        
        return torch.stack(batch_images), batch_labels

In [20]:
import torch.utils.data as data
from torch.autograd import Variable
import numpy as np


batch_size=100
train_loader= CustomDataset(f_train_data, final_train_lab, batch_size=batch_size, transform=transform_2)
test_loader = CustomDataset(f_test_data, final_test_lab, batch_size=batch_size, transform=transform_2)

print("Start training...")

correct=0
total=0
num_epochs = 3
for epoch in range(num_epochs):
    print(epoch)
    model.train()
    running_loss = 0.0
    for i in range(len(train_loader)):
        if i%10==0:
            print(i)
        
        images=train_loader[i][0].to(device)
        labels=train_loader[i][1].to(device)
        
        
        optimizer.zero_grad()
        outputs, _ = model(images, labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


        
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")
    print(f"Accuracy on training set: {100 * correct / total:.2f}%")

# Testing loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for i in range(len(test_loader)):
        images=test_loader[i][0].to(device)
        labels=test_loader[i][1].to(device)
        outputs, _ = model(images, labels)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")


350
training...
0
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
Epoch [1/3], Loss: 0.1616
Accuracy on training set: 95.33%
1
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
Epoch [2/3], Loss: 0.0890
Accuracy on training set: 96.29%
2
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
Epoch [3/3], Loss: 0.0616
Accuracy on training set: 96.90%
Accuracy on test set: 97.75%


"\nfor epoch in range(EPOCHS):        \n    for step, (x, y) in enumerate(train_loader):\n        # Change input array into list with each batch being one element\n        x = np.split(np.squeeze(np.array(x)), BATCH_SIZE)\n        # Remove unecessary dimension\n        for index, array in enumerate(x):\n            x[index] = np.squeeze(array)\n        # Apply feature extractor, stack back into 1 tensor and then convert to tensor\n        x = torch.tensor(np.stack(feature_extractor(x)['pixel_values'], axis=0))\n        # Send to GPU if available\n        x, y  = x.to(device), y.to(device)\n        b_x = Variable(x)   # batch x (image)\n        b_y = Variable(y)   # batch y (target)\n        # Feed through model\n        output, loss = model(b_x, None)\n        # Calculate loss\n        if loss is None: \n            loss = loss_func(output, b_y)   \n            optimizer.zero_grad()           \n            loss.backward()                 \n            optimizer.step()\n\n        if ste

In [22]:
torch.save(model.state_dict(), 'vit_mnist.pth')

In [24]:
import torch

# Embedding extraction
model.eval()  # Set the model to evaluation mode

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


def extract_embeddings(loader, model, device):
    all_embeddings = []
    all_labels = []
    
    with torch.no_grad():
        for i in range(len(loader)):
            if i%100==0:
                print(i)
            images=loader[i][0].to(device)
            labels=loader[i][1].to(device)
            embeddings=model.emb_extr_new_v1(pixel_values=images)
            all_embeddings.append(embeddings.cpu())
            all_labels.append(labels)
        
    all_embeddings = torch.cat(all_embeddings, dim=0) if all_embeddings else torch.tensor([])
    all_labels = torch.cat(all_labels, dim=0) if all_labels else torch.tensor([])
    return all_embeddings, all_labels


print("Training extraction...")
# Extract embeddings for training and test sets
train_embeddings, train_labels = extract_embeddings(train_loader, model, device)
print("Test extraction...")
test_embeddings, test_labels = extract_embeddings(test_loader, model, device)
print("done")


training extraction...
0
100
200
300
test extraction...
0
done


In [25]:
#load new unseen data and simulated drift data(label 8,9) in custom dataset

new_loader= CustomDataset(f_unseen_data, final_unseen_lab, batch_size=batch_size, transform=transform_2)
deg_loader = CustomDataset(f_deg_data, final_deg_lab, batch_size=batch_size, transform=transform_2)
print(len(train_loader))


print("New unseen extraction...")
# Extract embeddings for new unseen data and drift data
new_embeddings, new_labels = extract_embeddings(new_loader, model, device)
print("Drifted data extraction...")
deg_embeddings, deg_labels = extract_embeddings(deg_loader, model, device)
print("done")

350
new unseen extraction...
0
100
deg extraction...
0
100
done


In [26]:
def prediction_extraction(modello, dataloader):
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs, loss = modello(inputs, labels=None)
            _, predictions = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())
            
    return all_predictions, all_labels

def final_get_pred(model, data_loader):
    
    xx=[]
    with torch.no_grad():
        for i in range(len(data_loader)):

            images=data_loader[i][0].to(device)
            labels=data_loader[i][1].to(device)
            outputs, _ = model(images,labels)
            _, predicted = torch.max(outputs.data, 1)
            xx.append(predicted)

    new_xx=[]
    for i in range(len(xx)):
        for j in range(len(xx[i])):
            new_xx.append(xx[i][j])
    print(len(new_xx))
    
    return new_xx

# Extract the predictions of the model

print("train data predicted label prediction...")
train_pred=final_get_pred(model, train_loader)
print("test data predicted label prediction...")
test_pred=final_get_pred(model, test_loader)
print("new unseen data predicted label prediction...")
unseen_pred=final_get_pred(model, new_loader)
#print("drifted data predicted label prediction...")
#deg_pred=final_get_pred(model, deg_loader)


train pred extraction...
35000
test emb extraction...
8000
unseen emb extraction...
12000
deg emb extraction...


IndexError: Target 8 is out of bounds.

In [27]:
#since labels 8 and 9 are not available to the model we temporally
#label them as 0, the model is in evaluation mode so it won't update itself


temp_deg_labels=[]
tmp=final_test_lab[0]
print(type(tmp))
for i in range(len(deg_labels)):
    temp_deg_labels.append(0)

new_deg_loader = CustomDataset(f_deg_data, temp_deg_labels, batch_size=batch_size, transform=transform_2)


print("drifted data predicted label prediction......")
new_deg_pred=final_get_pred(model, new_deg_loader)

<class 'numpy.int64'>
deg emb extraction...
13000
so far so good...


In [29]:
import h5py

def save_embeddings(embedding, true_label,pr_label, path):
    fp = h5py.File(path, "w")
    fp.create_dataset("E", data=embedding, compression="gzip")    
    fp.create_dataset("Y_predicted", data=pr_label, compression="gzip") 
    fp.create_dataset("Y_original", data=true_label, compression="gzip") 
    fp.close()
    print("embeddings saved")

train_path="vit_MNIST_train_emb.hdf5"
test_path="vit_MNIST_test_emb.hdf5"
new_path="vit_MNIST_new_emb.hdf5"
deg_path="vit_MNIST_deg_emb.hdf5"

save_embeddings(train_embeddings, final_train_lab, train_pred, train_path )
save_embeddings(test_embeddings, final_test_lab, test_pred, test_path )
save_embeddings(new_embeddings, final_unseen_lab, unseen_pred, new_path)
save_embeddings(deg_embeddings, final_deg_lab, new_deg_pred, deg_path)
print("the end")

embeddings saved
embeddings saved
embeddings saved
embeddings saved
the end


In [ ]:
print(len(test_embeddings))
print(len(test_embeddings[0]))

In [15]:
def extract_transformer_embeddings(model, dataset):
    embeddings = []
    labels = []

    dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        for images, batch_labels in dataloader:
            # Forward pass through the model to obtain transformer encoder outputs
            _, transformer_outputs = model(images)

            # Append the transformer encoder outputs and corresponding labels
            embeddings.append(transformer_outputs)
            labels.append(batch_labels)

    # Concatenate embeddings and labels along the batch dimension
    embeddings = torch.cat(embeddings, dim=0)
    labels = torch.cat(labels, dim=0)

    return embeddings, labels


train_dataset = CustomDataset(f_train_data, final_train_lab, batch_size=batch_size, transform=transform)
test_dataset = CustomDataset(f_test_data, final_test_lab, batch_size=batch_size, transform=transform)

# Assuming 'model' is an instance of your VisionTransformer model
train_embeddings, train_labels = extract_transformer_embeddings(model, train_dataset)
test_embeddings, test_labels = extract_transformer_embeddings(model, test_dataset)
print(train_embeddings.shape)

ValueError: too many values to unpack (expected 4)

In [ ]:
def embedding_extraction(modello, dataloader):
    
    embeddings=[]
    print("extraction embedding...")

    with torch.no_grad():
        embeddings = []

        for images, _ in dataloader:  # Adjust this loop based on your data loading
            outputs = modello.emb_extr_new_v1(images)
            embeddings.append(outputs)
            
    embeddings=np.array(embeddings)
            
    return embeddings

def extract_embeddings_from_dataset(dataset, model):
    embeddings = []
    for i in range(len(dataset)):
        image, _ = dataset[i]
        embeddings.append(model.emb_extr_new_v1(image))
    return torch.stack(embeddings)


def into_dataloader(ds):
    dataloader = data.DataLoader(ds, batch_size=500, shuffle=False)
    
    return dataloader



model.eval()

# Extract embeddings from training and test data


# Perform inference on the test set
train_loader= CustomDataset(f_train_data, final_train_lab, batch_size=batch_size, transform=transform)
test_loader = CustomDataset(f_test_data, final_test_lab, batch_size=batch_size, transform=transform)
unseen_loader= CustomDataset(f_unseen_data, final_unseen_lab, batch_size=batch_size, transform=transform)
deg_loader = CustomDataset(f_deg_data, final_deg_lab, batch_size=batch_size, transform=transform)
print("train")
train_embeddings = extract_embeddings_from_dataset(train_loader, model)
print("test")
test_embeddings = extract_embeddings_from_dataset(test_loader, model)
print("new")
unseen_embeddings = extract_embeddings_from_dataset(unseen_loader, model)
print("drift")
deg_embeddings = extract_embeddings_from_dataset(deg_loader, model)
print("done")

train


In [ ]:
from torchvision import transforms
from PIL import Image

num_classes = 8  # Replace with the number of classes in your specific task

model.eval()

class CustomTestDataset():
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image, label = self.data[index]
        # Perform any necessary preprocessing on the image
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])
        image = transform(Image.fromarray(image))
        return image, label
    

#train_loader = data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)
#test_loader  = data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4) 
#unseen_loader = data.DataLoader(new_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)
#deg_loader  = data.DataLoader(deg_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4) 


# Create a DataLoader for the test dataset
test_dataset = CustomTestDataset(test_ds)
print(test_dataset[0])
test_dataloader = data.DataLoader(test_ds, batch_size=1000, shuffle=False)

# Perform inference on the test set
all_labels = []
all_predictions = []

print("evaluation...")
with torch.no_grad():
    for inputs, labels in test_dataloader:
        outputs, loss = model(inputs, labels)
        _, predictions = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())
        #all_embeddings.append(embeddings.cpu().numpy())

# Evaluate the model

print(confusion_matrix(all_labels, all_predictions))

# Print classification report
print("Classification Report:")
print(classification_report(all_labels, all_predictions))



In [22]:
def embedding_extraction(modello, dataloader):
    
    embeddings=[]
    print("extraction embedding...")

    with torch.no_grad():
        embeddings = []

        for images, _ in dataloader:  # Adjust this loop based on your data loading
            outputs = modello.emb_extr_new_v1(images)
            embeddings.append(outputs)
            
    embeddings=np.array(embeddings)
            
    return embeddings

def into_dataloader(ds):
    dataloader = data.DataLoader(ds, batch_size=1000, shuffle=False)
    
    return dataloader




model.eval()
# Perform inference on the test set


print("train")
train_emb=embedding_extraction(model, into_dataloader(train_ds))
print("test")
test_emb=embedding_extraction(model, into_dataloader(test_ds))
print("new")
new_emb=embedding_extraction(model, into_dataloader(new_ds))
print("drift")
deg_emb=embedding_extraction(model, into_dataloader(deg_ds))

print("done")

train
extraction embedding...


ValueError: Input image size (28*3) doesn't match model (224*224).

In [ ]:
def prediction_extraction(modello, dataloader):
    all_labels = []
    all_predictions = []

    print("inizio evaluation...")
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs, loss = modello(inputs, labels=None)
            _, predictions = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())
            
    return all_predictions, all_labels




print("predictions...")
print("train")
train_pred, train_lab=prediction_extraction(model, into_dataloader(train_ds))
print("test")
test_pred, test_lab=prediction_extraction(model, into_dataloader(test_ds))
print("new")
new_pred, new_lab=prediction_extraction(model, into_dataloader(new_ds))
print("drift")
deg_pred, deg_lab=prediction_extraction(model, into_dataloader(deg_ds))
print("done")

In [ ]:
print(len(train_emb))
print(len(train_lab))

print(len(train_emb[0]))
print(train_lab[0])

In [14]:
def de_batch(emb):
    final_e=[]
    for i in range(len(emb)):
        for j in range(len(emb[i])):
            final_e.append(emb[i][j])
    
    final_e=np.array(final_e, dtype='object')
    return final_e

print("debatching...")

final_train_emb=de_batch(train_emb)
final_test_emb=de_batch(test_emb)
final_new_emb=de_batch(new_emb)
final_deg_emb=de_batch(deg_emb)


debatching...


/tmp/ipykernel_2138420/1921475119.py:7: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  final_e=np.array(final_e, dtype='object')


In [15]:

def final_step(dati):
    temp=[]
    for i in range(len(dati)):
        if i%500==0:
            print(i)
        tmp=[]
        for j in range(len(dati[i])):
            tmp.append(dati[i][j])
        temp.append(np.array(tmp))
    
    temp=np.array(temp)
    
    return temp

print("train...")
fin_train_emb=final_step(final_train_emb)
print("test...")
fin_test_emb=final_step(final_test_emb)
print("new...")
fin_new_emb=final_step(final_new_emb)
print("deg...")
fin_deg_emb=final_step(final_deg_emb)


train...
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
test...
0
500
1000
1500
2000
2500
3000
3500
new...
0
500
1000
1500
2000
2500
3000
3500
4000
deg...
0
500
1000
1500
2000
2500


In [16]:
import h5py

def save_embeddings(embedding, true_label,pr_label, path):
    fp = h5py.File(path, "w")
    fp.create_dataset("E", data=embedding, compression="gzip")    
    fp.create_dataset("Y_predicted", data=pr_label, compression="gzip") 
    fp.create_dataset("Y_original", data=true_label, compression="gzip") 
    fp.close()
    print("embeddings saved")

train_path="vit_Intel_train_emb.hdf5"
test_path="vit_Intel_test_emb.hdf5"
new_path="vit_Intel_new_emb.hdf5"
deg_path="vit_Intel_deg_emb.hdf5"

save_embeddings(fin_train_emb, train_lab, train_pred, train_path )
save_embeddings(fin_test_emb, test_lab, test_pred, test_path )
save_embeddings(fin_new_emb, new_lab, new_pred, new_path)
save_embeddings(fin_deg_emb, deg_lab, deg_pred, deg_path)

embeddings saved
embeddings saved
embeddings saved
embeddings saved
